In [ ]:
"""
Best Practice Python Logger Implementation

This module provides a robust, configurable logging setup following best practices.
Features:
- Multiple output destinations (console, file, rotating files)
- Customizable log levels
- JSON formatting support for better log aggregation
- Context-based logging
- Correlation IDs for tracking request flows
- Singleton pattern for consistent logging across modules
"""

import logging
import os
import sys
import uuid
from datetime import datetime
from functools import wraps
from logging.handlers import RotatingFileHandler, TimedRotatingFileHandler
from pathlib import Path
from typing import Any, Callable, Dict, Optional
from pythonjsonlogger.jsonlogger import JsonFormatter
from functools import wraps

import structlog  # pip install structlog


class LoggerFactory:
    """
    Factory class for creating and configuring loggers.
    Implements the Singleton pattern to ensure consistent logging configuration.
    """
    _instance = None
    _initialized = False
    _loggers = {}
    _default_config = {
        'level': 'INFO',
        'format': '%(asctime)s - %(name)s - %(levelname)s - %(message)s',
        'json_format': '%(timestamp)s %(level)s %(name)s %(message)s',
        'date_format': '%Y-%m-%d %H:%M:%S',
        'log_dir': 'logs',
        'enable_console': True,
        'enable_file': True,
        'enable_json': False,
        'rotating_file': {
            'enabled': False,
            'max_bytes': 10485760,  # 10MB
            'backup_count': 5
        },
        'timed_rotating_file': {
            'enabled': False,
            'when': 'midnight',
            'interval': 1,
            'backup_count': 7
        }
    }

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(LoggerFactory, cls).__new__(cls)
        return cls._instance

    def __init__(self, config: Optional[Dict[str, Any]] = None):
        if not self._initialized:
            self._config = self._default_config.copy()
            if config:
                self._update_config(config)
            
            # Create log directory if needed
            if self._config['enable_file']:
                log_dir = Path(self._config['log_dir'])
                log_dir.mkdir(parents=True, exist_ok=True)
            
            self._initialized = True

    def _update_config(self, config: Dict[str, Any]) -> None:
        """Recursively update the configuration dictionary."""
        for key, value in config.items():
            if isinstance(value, dict) and key in self._config and isinstance(self._config[key], dict):
                self._config[key].update(value)
            else:
                self._config[key] = value

    def get_logger(self, name: str) -> 'AppLogger':
        """
        Get or create a logger with the given name.
        
        Args:
            name: The name of the logger.
            
        Returns:
            An AppLogger instance.
        """
        if name not in self._loggers:
            # Create the logger
            logger = logging.getLogger(name)
            
            # Configure level
            logger.setLevel(getattr(logging, self._config['level']))
            
            # Clear any existing handlers
            if logger.handlers:
                logger.handlers.clear()
            
            # Add handlers based on configuration
            self._add_handlers(logger, name)
            
            # Wrap with AppLogger for extra functionality
            self._loggers[name] = AppLogger(logger)
        
        return self._loggers[name]

    def _add_handlers(self, logger: logging.Logger, name: str) -> None:
        """
        Add handlers to the logger based on the configuration.
        
        Args:
            logger: The logger to add handlers to.
            name: The name of the logger (used for file naming).
        """
        # Console handler
        if self._config['enable_console']:
            console_handler = logging.StreamHandler(sys.stdout)
            console_formatter = self._create_formatter(False)
            console_handler.setFormatter(console_formatter)
            logger.addHandler(console_handler)
        
        # Regular file handler
        if self._config['enable_file'] and not (
            self._config['rotating_file']['enabled'] or 
            self._config['timed_rotating_file']['enabled']
        ):
            file_handler = logging.FileHandler(
                os.path.join(self._config['log_dir'], f"{name}.log")
            )
            file_formatter = self._create_formatter(self._config['enable_json'])
            file_handler.setFormatter(file_formatter)
            logger.addHandler(file_handler)
        
        # Rotating file handler
        if self._config['enable_file'] and self._config['rotating_file']['enabled']:
            rotating_handler = RotatingFileHandler(
                os.path.join(self._config['log_dir'], f"{name}.log"),
                maxBytes=self._config['rotating_file']['max_bytes'],
                backupCount=self._config['rotating_file']['backup_count']
            )
            rotating_formatter = self._create_formatter(self._config['enable_json'])
            rotating_handler.setFormatter(rotating_formatter)
            logger.addHandler(rotating_handler)
        
        # Timed rotating file handler
        if self._config['enable_file'] and self._config['timed_rotating_file']['enabled']:
            timed_handler = TimedRotatingFileHandler(
                os.path.join(self._config['log_dir'], f"{name}.log"),
                when=self._config['timed_rotating_file']['when'],
                interval=self._config['timed_rotating_file']['interval'],
                backupCount=self._config['timed_rotating_file']['backup_count']
            )
            timed_formatter = self._create_formatter(self._config['enable_json'])
            timed_handler.setFormatter(timed_formatter)
            logger.addHandler(timed_handler)

    def _create_formatter(self, use_json: bool) -> logging.Formatter:
        """
        Create a formatter based on the configuration.
        
        Args:
            use_json: Whether to use JSON formatting.
            
        Returns:
            A logging formatter instance.
        """
        if use_json:
            return JsonFormatter(
                fmt=self._config['json_format'],
                datefmt=self._config['date_format']
            )
        else:
            return logging.Formatter(
                fmt=self._config['format'],
                datefmt=self._config['date_format']
            )

    def configure_structlog(self) -> None:
        """Configure structlog for structured logging."""
        structlog.configure(
            processors=[
                structlog.stdlib.filter_by_level,
                structlog.stdlib.add_logger_name,
                structlog.stdlib.add_log_level,
                structlog.stdlib.PositionalArgumentsFormatter(),
                structlog.processors.TimeStamper(fmt="iso"),
                structlog.processors.StackInfoRenderer(),
                structlog.processors.format_exc_info,
                structlog.processors.UnicodeDecoder(),
                structlog.stdlib.render_to_log_kwargs,
            ],
            context_class=dict,
            logger_factory=structlog.stdlib.LoggerFactory(),
            wrapper_class=structlog.stdlib.BoundLogger,
            cache_logger_on_first_use=True,
        )


class AppLogger:
    """
    Enhanced logger wrapper that adds additional functionality:
    - Context-based logging
    - Correlation IDs for tracking request flows
    - Exception logging with proper traceback
    - Method entry/exit logging (useful for debugging and performance monitoring)
    """
    
    def __init__(self, logger: logging.Logger):
        self._logger = logger
        self._context = {}

    def get_log_correlation_id(self) -> str:
        if 'correlation_id' in self._context:
            return self._context['correlation_id']
        else:
            return None
    
    def with_context(self, **kwargs) -> 'AppLogger':
        """
        Create a new logger with additional context.
        
        Args:
            **kwargs: Context key-value pairs to add.
            
        Returns:
            A new AppLogger instance with the combined context.
        """
        new_logger = AppLogger(self._logger)
        new_logger._context = {**self._context, **kwargs}
        return new_logger
    
    def with_correlation_id(self, correlation_id: Optional[str] = None) -> 'AppLogger':
        """
        Add a correlation ID to the logger context.
        
        Args:
            correlation_id: The correlation ID to use. If None, a new UUID will be generated.
            
        Returns:
            A new AppLogger instance with the correlation ID in the context.
        """
        if correlation_id is None:
            correlation_id = str(uuid.uuid4())
        return self.with_context(correlation_id=correlation_id)
    
    def _format_message(self, message: str) -> str:
        """
        Format the message with the context if available.
        
        Args:
            message: The original message.
            
        Returns:
            The formatted message with context.
        """
        if not self._context:
            return message
        
        context_str = ' '.join(f"{k}={v}" for k, v in self._context.items())
        return f"{message} [{context_str}]"
    
    def debug(self, message: str, *args, **kwargs) -> None:
        """Log a debug message."""
        self._logger.debug(self._format_message(message), *args, **kwargs)
    
    def info(self, message: str, *args, **kwargs) -> None:
        """Log an info message."""
        self._logger.info(self._format_message(message), *args, **kwargs)
    
    def warning(self, message: str, *args, **kwargs) -> None:
        """Log a warning message."""
        self._logger.warning(self._format_message(message), *args, **kwargs)
    
    def error(self, message: str, *args, exc_info: Optional[Exception] = None, **kwargs) -> None:
        """
        Log an error message with optional exception info.
        
        Args:
            message: The error message.
            *args: Additional positional arguments.
            exc_info: Optional exception to include in the log.
            **kwargs: Additional keyword arguments.
        """
        self._logger.error(
            self._format_message(message), 
            *args, 
            exc_info=exc_info or kwargs.pop('exc_info', None), 
            **kwargs
        )
    
    def critical(self, message: str, *args, exc_info: Optional[Exception] = None, **kwargs) -> None:
        """
        Log a critical message with optional exception info.
        
        Args:
            message: The critical message.
            *args: Additional positional arguments.
            exc_info: Optional exception to include in the log.
            **kwargs: Additional keyword arguments.
        """
        self._logger.critical(
            self._format_message(message), 
            *args, 
            exc_info=exc_info or kwargs.pop('exc_info', None), 
            **kwargs
        )
    
    def exception(self, message: str, *args, **kwargs) -> None:
        """
        Log an exception message with the current exception info.
        
        Args:
            message: The exception message.
            *args: Additional positional arguments.
            **kwargs: Additional keyword arguments.
        """
        self._logger.exception(self._format_message(message), *args, **kwargs)
    
    def log_method_call(self, method_name: str, *args, **kwargs) -> None:
        """
        Log a method call with its arguments.
        
        Args:
            method_name: The name of the method being called.
            *args: The method arguments.
            **kwargs: The method keyword arguments.
        """
        args_str = ', '.join(repr(arg) for arg in args)
        kwargs_str = ', '.join(f"{k}={repr(v)}" for k, v in kwargs.items())
        all_args = ', '.join(filter(None, [args_str, kwargs_str]))
        self.debug(f"Called {method_name}({all_args})")


def method_logger(level: str = 'DEBUG') -> Callable:
    """
    Decorator to log method entry, exit, timing, arguments, return values, and exceptions.

    Args:
        level (str): Logging level (e.g., 'DEBUG', 'INFO').

    Returns:
        Callable: Wrapped function with enhanced logging.
    """
    log_level = getattr(logging, level.upper(), logging.DEBUG)

    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            # Infer logger name based on class or module context
            if args and hasattr(args[0], '__class__'):
                logger_name = f"{args[0].__class__.__module__}.{args[0].__class__.__name__}"
            else:
                logger_name = func.__module__

            logger = LoggerFactory().get_logger(logger_name)
            func_name = func.__name__

            # Log entry with arguments
            logger._logger.log(
                log_level,
                f"Entering {func_name} with args={args}, kwargs={kwargs}"
            )

            start_time = datetime.now()
            try:
                result = func(*args, **kwargs)
                execution_time = (datetime.now() - start_time).total_seconds()

                # Log successful exit
                logger._logger.log(
                    log_level,
                    f"Exiting {func_name} with result={result!r} "
                    f"(took {execution_time:.3f}s)"
                )
                return result

            except Exception as e:
                # Log error with traceback and time
                execution_time = (datetime.now() - start_time).total_seconds()
                logger.error(
                    f"Exception in {func_name} (took {execution_time:.3f}s): {str(e)}",
                    exc_info=e
                )
                raise

        return wrapper
    return decorator


# Factory function to get a logger
def get_logger(name: str, config: Optional[Dict[str, Any]] = None) -> AppLogger:
    """
    Get a logger instance with the given name and optional configuration.
    
    Args:
        name: The name of the logger.
        config: Optional configuration dictionary to override defaults.
        
    Returns:
        An AppLogger instance.
    """
    factory = LoggerFactory(config)
    return factory.get_logger(name)


# Example usage with structlog integration
def get_structlog_logger(name: str, config: Optional[Dict[str, Any]] = None) -> structlog.BoundLogger:
    """
    Get a structlog logger instance with the given name and optional configuration.
    
    Args:
        name: The name of the logger.
        config: Optional configuration dictionary to override defaults.
        
    Returns:
        A structlog.BoundLogger instance.
    """
    factory = LoggerFactory(config)
    factory.configure_structlog()
    return structlog.get_logger(name)

# # Demonstrating the method_logger decorator
# @method_logger(level='INFO')
# def sample_function(x, y):
#     print(f"Calculating {x} + {y}")
#     return x + y

# Example usage
def example_usage():
    # Basic usage
    logger = get_logger("example")
    logger.info("This is a simple log message")
    
    # Using context
    context_logger = logger.with_context(user_id="12345", session_id="abc123")
    context_logger.info("User logged in")
    
    # Using correlation ID for request tracking
    request_logger = logger.with_correlation_id()
    request_logger.info(f"Processing request, correlation_id: {request_logger.get_log_correlation_id()}")
    
    # Logging exceptions
    try:
        result = 1 / 0
    except Exception as e:
        logger.error("An error occurred during calculation", exc_info=e)
    
    # Advanced configuration
    advanced_config = {
        'level': 'DEBUG',
        'enable_json': True,
        'enable_console': True,  # Ensure console output works too for debugging
        'rotating_file': {
            'enabled': True,
            'max_bytes': 5242880,  # 5MB
            'backup_count': 3
        }
    }

    # Create and test the logger
    advanced_logger = get_logger("advanced_example", advanced_config)

    # Force correct level on both logger and handlers
    advanced_logger._logger.setLevel(logging.DEBUG)
    for handler in advanced_logger._logger.handlers:
        handler.setLevel(logging.DEBUG)
        if hasattr(handler, 'baseFilename'):
            print(f"Writing to log file: {handler.baseFilename}")

    # Ensure propagation doesn't interfere
    advanced_logger._logger.propagate = False

    # Log messages at different levels
    advanced_logger.debug("Debug message")
    advanced_logger.info("Info message")
    advanced_logger.warning("Warning message")
    advanced_logger.error("Error message")

    # Force flush all handlers
    for handler in advanced_logger._logger.handlers:
        if hasattr(handler, 'flush'):
            handler.flush()

    # Shutdown logging system to flush buffers
    logging.shutdown()

    return True